In [1]:
import os
import time
import yaml
import math
import pandas as pd
import numpy as np

In [2]:
from data.excel2pd import data_load
from data.preprocessing import data_preprocessing
from data.train_test_split import data2Window, X_Y2train_test, X_Y_from_data
from data.CustomDataset import CustomDataset

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더
from model.model import seq2seq

In [4]:
f = open("Settings.yaml", 'r')
setup = yaml.load(f, Loader=yaml.FullLoader)

data = data_load(11)
scaler, data = data_preprocessing(data, setup['data_options'])

X, Y = X_Y_from_data(data)

X_train, Y_train, X_test, Y_test = X_Y2train_test(X, Y,
                                                  setup['input_window'],
                                                  setup['output_window'],
                                                  setup['stride'])

In [5]:
dataset = CustomDataset(X_train, Y_train)
dataloader = DataLoader(dataset, batch_size = setup['batch'], shuffle = False)

In [6]:
model = seq2seq(input_size = 3, hidden_size = setup['LSTM'][0], num_layers = 1, dropout = setup['dropout'])

c:\ProgramData\Anaconda3\envs\torch\lib\site-packages\torch\nn\modules\rnn.py:67: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [7]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [9]:
model.apply(init_weights)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 680 trainable parameters


In [10]:
model.train()

seq2seq(
  (encoder): Encoder(
    (lstm): LSTM(3, 7, batch_first=True, dropout=0.5)
  )
  (decoder): Decoder(
    (lstm): LSTM(3, 7, batch_first=True, dropout=0.5)
    (output_layer): Linear(in_features=7, out_features=1, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)